# Multi-Domain GRPO Training with Gemma 3 (1B) using Tunix

This notebook presents a **complete, end-to-end training pipeline** for teaching a **Gemma 3 (1B)** language model to **explicitly show its reasoning** before producing a final answer.

The core objective is not just answer accuracy, but **reasoning transparency**.  
The model is trained to consistently generate outputs in the following structured format:

```

<reasoning>
step-by-step reasoning trace
</reasoning>
<answer>
final answer
</answer>
```

Training is performed using **Group Relative Policy Optimization (GRPO)** via **Tunix**, Google’s JAX-native post-training library. GRPO enables stable reinforcement-learning–style updates without requiring a separate value model, making it well-suited for **small models and limited TPU budgets**.

---

## What This Notebook Demonstrates

* Fine-tuning **Gemma 3 (1B)** on a **single Kaggle TPU session**
* Multi-domain reasoning training across:

  * Math
  * Logic
  * Basic science
  * Coding (non-executed, verifiable by structure)
* Strict enforcement of **reasoning + answer separation**
* Reward shaping for:

  * Format compliance
  * Refusal avoidance
  * Correctness (where applicable)
  * Conciseness and termination discipline
* A **phase-based curriculum** inspired by Open-R1 / R1-style training

This notebook is intentionally written to be:

* **Minimal** (no YAML, no hidden configs)
* **Debuggable** (explicit reward functions)
* **Reproducible** (single-session training)
* **Notebook-native** (no external orchestration)

---

## Scope and Constraints

* **Model**: Gemma 3 (1B-IT)
* **Framework**: Tunix (JAX)
* **Hardware**: Kaggle TPU (single session)
* **Max output length**: < 1K tokens
* **Language**: English only
* **No tool use or code execution during training**

Coding tasks are evaluated **without running unit tests**. Instead, the model is rewarded for producing **well-formed, logically correct Python code** that follows the required format, aligning with the hackathon’s LLM-as-a-judge evaluation setup.

---

## Why This Matters

Large reasoning models are expensive to train.
This notebook shows that **even a 1B-parameter model** can be taught to reason more transparently using:

* Careful reward design
* Curriculum-style training
* Strong format constraints

The result is a small, efficient model that **thinks before it answers**—making reasoning more accessible, interpretable, and reproducible.

In [ ]:
import os
os.environ["HF_HUB_DISABLE_XET"] = "1"

## Install necessary libraries

In [ ]:
!pip install -q kagglehub

!pip install -q ipywidgets

!pip install -q tensorflow
!pip install -q tensorflow_datasets
!pip install -q tensorboardX
!pip install -q transformers
!pip install -q grain
# !pip install "google-tunix[prod]==0.1.5"

!pip install -q git+https://github.com/google/tunix
!pip install -q git+https://github.com/google/qwix

!pip uninstall -q -y flax
# !pip install -U flax
!pip install flax==0.12.0

!pip install -q datasets wandb==0.22.0

In [ ]:
import wandb, os
from kaggle_secrets import UserSecretsClient
os.environ['WANDB_API_KEY'] = UserSecretsClient().get_secret("WANDB_API_KEY")

## Imports

In [ ]:
import functools
import gc
import os
from pprint import pprint
import re

import csv
import shutil

from flax import nnx
import grain
import humanize
import jax
import jax.numpy as jnp
import kagglehub
import optax
from orbax import checkpoint as ocp
from pathlib import Path
import qwix
import tensorflow_datasets as tfds
from tqdm.auto import tqdm
from tunix.generate import sampler as sampler_lib
from tunix.generate import tokenizer_adapter as tokenizer_lib
from tunix.models.gemma3 import params
from tunix.models.gemma3 import model
from tunix.rl import rl_cluster as rl_cluster_lib
from tunix.rl.grpo.grpo_learner import GRPOConfig, GRPOLearner
from tunix.rl.rollout import base_rollout
from tunix.sft import metrics_logger
from datasets import load_dataset

## Hyperparameters

Let's define the configuration we are going to use. Note that this is by no
means a "perfect" set of hyperparameters. To get good results, you might have
to train the model for longer.

In [ ]:
# ====== Data ======
TRAIN_DATA_DIR = "./data/train"
TEST_DATA_DIR = "./data/test"
TRAIN_FRACTION = 1.0

# ====== LoRA ======
RANK = 64
ALPHA = 64.0

# ====== Sharding ======
MESH = [(1, 4), ("fsdp", "tp")]

# ====== GRPO ======
# === Generation during GRPO training ===
MAX_PROMPT_LENGTH = 256
TOTAL_GENERATION_STEPS = 512
# Important to keep a high-ish temperature for varied, diverse responses during
# training.
TEMPERATURE = 0.9
TOP_P = 1.0
TOP_K = 50
# The number of times the policy generates multiple responses for a given prompt
# within a single training step. This corresponds to `G` in Algorithm 1 in the
# paper. The "group" in GRPO comes from here.
NUM_GENERATIONS = 4

# === other GRPO configs ===
# The number of iterations per batch (𝜇 in GRPO algo 1).
NUM_ITERATIONS = 1
# The coefficient for the KL divergence penalty (𝛽) in the GRPO loss function.
# Important to keep a high enough value for this, otherwise, the KL divergence
# can increase unchecked.
BETA = 0.08
# Epsilon value for clipping (𝜀 in GRPO loss in paper). Similar to PPO, for
# stable updates.
EPSILON = 0.2

# ====== Training ======
TRAIN_MICRO_BATCH_SIZE = 4
# Increase `NUM_BATCHES` and `MAX_STEPS` for better results.
# NUM_BATCHES = 3738
NUM_BATCHES = 3
# Keep `NUM_TEST_BATCHES` low so that evaluation runs quickly. It can be
# increased to a max. of 330 (if batch size is 4).
NUM_TEST_BATCHES = 100

EVAL_EVERY_N_STEPS = 10  # this doesn't matter if `TRAIN_FRACTION = 1.0`.
NUM_EPOCHS = 1  # can potentially train for more epochs

# Number of training steps.
MAX_STEPS = int(NUM_BATCHES * NUM_ITERATIONS * TRAIN_FRACTION * NUM_EPOCHS)

# === AdamW, warmup, cosine scheduler ===
LEARNING_RATE = 3e-6
B1 = 0.9
B2 = 0.99
WEIGHT_DECAY = 0.1
# == Cosine decay with warmup scheduler ==
# Linearly increase learning rate from 0. to 5e-6 in the first 10% training
# steps, and then gradually decrease the learning rate to 0 using cosine
# scheduler.
WARMUP_STEPS = 0.1 * MAX_STEPS
# == Grad clipping ==
# Grad clipping to prevent large gradients. Found this
# important to keep KL divergence in check.
MAX_GRAD_NORM = 0.1

# Checkpoint saving
INTERMEDIATE_CKPT_DIR = "/tmp/content/intermediate_ckpt/"
CKPT_DIR = "/tmp/content/ckpts/"
SAVE_INTERVAL_STEPS = 500
MAX_TO_KEEP = 4

# ====== Inference ======
GENERATION_CONFIGS = {
    # greedy search
    "greedy": {"temperature": 1e-4, "top_k": 1, "top_p": 1.0},
    # some randomness
    "standard": {"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    # liberal
    "liberal": {"temperature": 0.85, "top_k": 2000, "top_p": 1.0},
}

## Utility functions

In [ ]:
def show_hbm_usage():
  """Displays memory usage per device."""
  fmt_size = functools.partial(humanize.naturalsize, binary=True)

  for d in jax.local_devices():
    stats = d.memory_stats()
    used = stats["bytes_in_use"]
    limit = stats["bytes_limit"]
    print(f"Using {fmt_size(used)} / {fmt_size(limit)} ({used/limit:%}) on {d}")

## Data preprocessing

First, let's define some special tokens. We instruct the model to first reason
between the `<reasoning>` and `</reasoning>` tokens. After
reasoning, we expect it to provide the answer between the `<answer>` and
`</answer>` tokens.

In [ ]:
REASONING_START = "<reasoning>"
REASONING_END = "</reasoning>"
ANSWER_START = "<answer>"
ANSWER_END = "</answer>"

SYSTEM_PROMPT = f"""
You are given a problem.

Think step by step and write your reasoning between
{REASONING_START} and {REASONING_END}.

Then write the final answer as a single value between
{ANSWER_START} and {ANSWER_END}.

Do not write anything outside these tags.
""".strip()

TEMPLATE = """<start_of_turn>user
{system_prompt}

{question}<end_of_turn>
<start_of_turn>model"""

We use OpenAI's [GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k), which comprises grade school math word problems.

In [ ]:
from datasets import load_dataset
import grain
import os
import time

# =========================
# GSM8K answer extractor
# =========================
def extract_hash_answer(text: str) -> str | None:
    if not isinstance(text, str):
        return None
    if "####" not in text:
        return None
    return text.split("####")[-1].strip()


# =========================
# MBPP test_list fixer
# =========================
def build_mbpp_tests(x):
    tests = x.get("test_list", [])
    if not isinstance(tests, list):
        return []

    test_cases = []
    for t in tests:
        if isinstance(t, dict) and "input" in t and "output" in t:
            test_cases.append({
                "input": t["input"],
                "output": t["output"],
            })
    return test_cases


# =========================
# DATASET BUILDER
# =========================
def get_dataset(split="train") -> grain.MapDataset:
    print(f"\n🚀 Building mixed dataset | split = {split}")
    t0 = time.time()

    os.environ["HF_HUB_DISABLE_XET"] = "1"
    os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

    # ============
    # GSM8K (MATH)
    # ============
    print("📥 Loading GSM8K...")
    gsm = load_dataset("openai/gsm8k", "main", split=split)
    print(f"✅ GSM8K loaded ({len(gsm)} samples)")

    gsm_ds = (
        grain.MapDataset.source(gsm)
        .map(lambda x: {
            "prompts": TEMPLATE.format(
                system_prompt=SYSTEM_PROMPT,
                question=str(x["question"]),
            ),
            "question": str(x["question"]),
            "answer": extract_hash_answer(str(x["answer"])),
            "domain": "math",
        })
        .filter(lambda x: x["answer"] is not None)
    )

    print("🧠 GSM8K pipeline built")

    # ============
    # MBPP (CODE) — FIXED
    # ============
    print("📥 Loading MBPP...")
    mbpp = load_dataset("google-research-datasets/mbpp", "sanitized", split=split)
    print(f"✅ MBPP loaded ({len(mbpp)} samples)")

    def mbpp_map(x):
        test_cases = build_mbpp_tests(x)
        if not test_cases:
            return None  # 🚨 skip broken samples safely

        return {
            "prompts": TEMPLATE.format(
                system_prompt=SYSTEM_PROMPT,
                question=str(x["text"]),
            ),
            "question": str(x["text"]),
            "answer": None,
            "verification_info": {
                "language": "python",
                "test_cases": test_cases,
            },
            "domain": "code",
        }

    mbpp_ds = (
        grain.MapDataset.source(mbpp)
        .map(mbpp_map)
        .filter(lambda x: x is not None)
    )

    print(mbpp_ds[:1])
    print("🧠 MBPP pipeline built (SAFE)")

    # ============
    # ARC Easy (SCIENCE)
    # ============
    print("📥 Loading ARC Easy...")
    arc = load_dataset("allenai/ai2_arc", "ARC-Easy", split=split)
    print(f"✅ ARC Easy loaded ({len(arc)} samples)")

    def arc_prompt(x):
        choices = "\n".join(
            f"{l}. {t}"
            for l, t in zip(x["choices"]["label"], x["choices"]["text"])
        )
        return f"{x['question']}\n\nChoices:\n{choices}"

    arc_ds = grain.MapDataset.source(arc).map(lambda x: {
        "prompts": TEMPLATE.format(
            system_prompt=SYSTEM_PROMPT,
            question=arc_prompt(x),
        ),
        "question": x["question"],
        "answer": x["answerKey"],
        "domain": "science",
    })

    print("🧠 ARC pipeline built")

    # =================
    # StrategyQA (LOGIC)
    # =================
    print("📥 Loading StrategyQA...")
    strategyqa = load_dataset("ChilleD/StrategyQA", split=split)
    print(f"✅ StrategyQA loaded ({len(strategyqa)} samples)")

    def strategyqa_prompt(x):
        return f"Question: {x['question']}\nAnswer yes or no."

    strategyqa_ds = grain.MapDataset.source(strategyqa).map(lambda x: {
        "prompts": TEMPLATE.format(
            system_prompt=SYSTEM_PROMPT,
            question=strategyqa_prompt(x),
        ),
        "question": x["question"],
        "answer": "yes" if x["answer"] else "no",
        "domain": "logic",
    })

    print("🧠 StrategyQA pipeline built")

    # ============
    # MIX
    # ============
    print("🧪 Mixing datasets...")
    mixed = (
        grain.MapDataset.mix(
            datasets=[gsm_ds, mbpp_ds, arc_ds, strategyqa_ds],
            weights=[0.35, 0.30, 0.20, 0.15],
        )
        .shuffle(seed=42)
    )

    print("✅ Mixed dataset pipeline ready")
    print(f"⏱️ Total build time: {time.time() - t0:.2f}s")

    return mixed

We split the dataset set into train and test sets as usual.

In [ ]:
print("Using data source: huggingface (mixed domains)")

import os
import time
import grain
import itertools
from collections import Counter

os.environ["HF_HUB_DISABLE_XET"] = "1"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

t0 = time.time()

# =====================
# TRAIN DATASET (STREAMING)
# =====================
print("\n🚧 Building TRAIN dataset (STREAMING)...")
train_ds_raw = get_dataset("train")

print("🧪 Sampling ONE TRAIN example (safe)...")
sample = next(iter(train_ds_raw))
print("✅ TRAIN sample loaded")
print("   domain:", sample["domain"])
print("   question (truncated):", sample["question"][:80])

# ---- DEBUG: domain mix (small prefix only)
print("\n🔍 Checking TRAIN domain mix (first 300 samples)...")
train_domain_counter = Counter(
    ex["domain"] for ex in itertools.islice(train_ds_raw, 3000)
)
print("   Domain counts:", train_domain_counter)

# ---- STREAMING BATCHED DATASET (THIS IS WHAT GRPO USES)
print("\n📦 Creating STREAMING TRAIN batches...")
train_dataset_for_grpo = (
    train_ds_raw
    .to_iter_dataset()
    .batch(TRAIN_MICRO_BATCH_SIZE)
)

print("✅ TRAIN streaming dataset ready")

# =====================
# TEST DATASET (STREAMING)
# =====================
print("\n🚧 Building TEST dataset (STREAMING)...")
test_ds_raw = get_dataset("test")

print("🧪 Sampling ONE TEST example (safe)...")
sample = next(iter(test_ds_raw))
print("✅ TEST sample loaded")
print("   domain:", sample["domain"])
print("   question (truncated):", sample["question"][:80])

# ---- DEBUG: domain mix
print("\n🔍 Checking TEST domain mix (first 300 samples)...")
test_domain_counter = Counter(
    ex["domain"] for ex in itertools.islice(test_ds_raw, 300)
)
print("   Domain counts:", test_domain_counter)

# ---- STREAMING BATCHED TEST DATASET
print("\n📦 Creating STREAMING TEST batches...")
test_dataset_for_grpo = (
    test_ds_raw
    .to_iter_dataset()
    .batch(TRAIN_MICRO_BATCH_SIZE)
)

print("✅ TEST streaming dataset ready")

# =====================
# SUMMARY
# =====================
print("\n📊 DATASET STATUS")
print("   TRAIN: pure streaming (MBPP preserved)")
print("   TEST:  pure streaming (MBPP preserved)")
print(f"⏱️ Total setup time: {time.time() - t0:.2f}s")

Let's see how one batch of the training dataset looks like!


In [ ]:
from collections import Counter
from pprint import pprint

domain_counter = Counter()

N = 50  # number of batches to inspect
for i in range(N):
    batch = train_dataset[i]
    domains = batch["domain"]   # this is an array/list
    for d in domains:
        domain_counter[str(d)] += 1

print("Domain counts:")
pprint(domain_counter)

In [ ]:
for ele in train_dataset[:1]:
  pprint(ele)

## Load the policy model and the reference model

The policy model is the model which is actually trained and whose weights are
updated. The reference model is the model with which we compute KL divergence.
This is to ensure that the policy updates are not huge and that it does not
deviate too much from the reference model.

Typically, the reference model is the base model, and the policy model is the
same base model, but with LoRA parameters. Only the LoRA parameters are updated.

Note: We perform full precision (fp32) training. You can, however, leverage
Qwix for QAT.

To load the model, you need to be on [Kaggle](https://www.kaggle.com/) and need
to have agreed to the Gemma license
[here](https://www.kaggle.com/models/google/gemma/flax/).

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

os.environ["KAGGLE_KEY"] = user_secrets.get_secret("KAGGLE_KEY")
os.environ["KAGGLE_USERNAME"] = user_secrets.get_secret("KAGGLE_USERNAME")

# Now this will NOT trigger login
if "KAGGLE_USERNAME" not in os.environ or "KAGGLE_KEY" not in os.environ:
    kagglehub.login()

This code snippet serves as a workaround to re-save the pre-trained model checkpoint from Kaggle into a local format that is compatible with the [Flax NNX](https://flax.readthedocs.io/en/stable/why.html) library. Because the original checkpoint has parameter names and tensor structures that don't match the target NNX model architecture, it cannot be loaded directly.

We first load the original weights into a temporary model instance, then extract and re-save the model's state into a new, properly formatted local checkpoint, which can then be successfully loaded by the final sharded NNX model.

In [ ]:
from tunix.models.gemma3 import params
from tunix.models.gemma3 import model as gemma_model

import jax
import jax.numpy as jnp
import orbax.checkpoint as ocp
from flax import nnx

def get_gemma_ref_model(ckpt_path):
    # ===============================
    # Device mesh
    # ===============================
    mesh = jax.make_mesh(*MESH)

    # ===============================
    # Model config (✅ CORRECT API)
    # ===============================
    model_config = gemma_model.ModelConfig.gemma3_1b_it()

    # ===============================
    # Build abstract (shape-only) model
    # ===============================
    abs_gemma: nnx.Module = nnx.eval_shape(
        lambda: params.create_model_from_checkpoint(
            params.GEMMA3_1B_IT,
            model_config,
        )
    )

    # ===============================
    # Prepare sharded state structure
    # ===============================
    abs_state = nnx.state(abs_gemma)
    pspecs = nnx.get_named_sharding(abs_state, mesh)

    abs_state = jax.tree.map(
        lambda a, s: jax.ShapeDtypeStruct(
            a.shape,
            jnp.bfloat16,
            sharding=s,
        ),
        abs_state,
        pspecs,
    )

    # ===============================
    # Restore checkpoint
    # ===============================
    checkpointer = ocp.StandardCheckpointer()
    restored_params = checkpointer.restore(
        ckpt_path,
        target=abs_state,
    )

    # ===============================
    # Materialize reference model
    # ===============================
    graph_def, _ = nnx.split(abs_gemma)
    ref_model = nnx.merge(graph_def, restored_params)

    return ref_model, mesh, model_config


def get_lora_model(base_model, mesh):
    # ===============================
    # LoRA configuration
    # ===============================
    lora_provider = qwix.LoraProvider(
        module_path=(
            ".*q_einsum|.*kv_einsum|.*gate_proj|"
            ".*down_proj|.*up_proj|.*attn_vec_einsum"
        ),
        rank=RANK,
        alpha=ALPHA,
    )

    # ===============================
    # Apply LoRA
    # ===============================
    model_input = base_model.get_model_input()
    lora_model = qwix.apply_lora_to_model(
        base_model,
        lora_provider,
        **model_input,
    )

    # ===============================
    # Re-apply sharding
    # ===============================
    with mesh:
        state = nnx.state(lora_model)
        pspecs = nnx.get_partition_spec(state)
        sharded_state = jax.lax.with_sharding_constraint(state, pspecs)
        nnx.update(lora_model, sharded_state)

    return lora_model

In [ ]:
# ===============================
# Cleanup
# ===============================
!rm -rf /tmp/content/intermediate_ckpt/*
!rm -rf /tmp/content/ckpts/*

import os, gc, jax
import jax.numpy as jnp
from tunix.models.gemma3 import params
from tunix.models.gemma3 import model as gemma_model
from flax import nnx
import orbax.checkpoint as ocp

CKPT_PATH = os.path.join(INTERMEDIATE_CKPT_DIR, "state")

# ===============================
# Model config (CORRECT)
# ===============================
model_config = gemma_model.ModelConfig.gemma3_1b_it()

# ===============================
# Load base Gemma 3 1B
# ===============================
base_model = params.create_model_from_checkpoint(
    params.GEMMA3_1B_IT,
    model_config,
)

tokenizer = params.create_tokenizer()
print("✅ Base Gemma-3 1B loaded")

# ===============================
# Save clean base state
# ===============================
checkpointer = ocp.StandardCheckpointer()
_, base_state = nnx.split(base_model)

checkpointer.save(CKPT_PATH, base_state)
checkpointer.wait_until_finished()

print("✅ Clean base checkpoint saved")

### Model Loading and LoRA Application

These two functions work together to load a base model from a checkpoint and apply a LoRA (Low-Rank Adaptation) layer to it.

* `get_ref_model`: Loads the complete Gemma model from a specified checkpoint path. It uses **JAX sharding** to distribute the model parameters across multiple devices.
* `get_lora_model`: Takes the base model and applies LoRA layers to it. It uses a `LoraProvider` to select specific layers (like attention and MLP layers) to be adapted. The resulting LoRA-infused model is then sharded and updated to ensure it's ready for distributed training.

Now we load reference and policy Gemma models using the Flax NNX library and display their structures.

In [ ]:
# ===============================
# Cell 1: Load HF Qwen3-1.7B
# ===============================
import os, gc, torch
from transformers import AutoModelForCausalLM

HF_QWEN_PATH = "/kaggle/input/qwen-3/transformers/1.7b-base/1"

print("📥 Loading HF Qwen3-1.7B...")
hf_model = AutoModelForCausalLM.from_pretrained(
    HF_QWEN_PATH,
    torch_dtype=torch.float16,
    device_map="cpu",
    trust_remote_code=True,
)

hf_state = hf_model.state_dict()
print(f"✅ HF model loaded with {len(hf_state)} tensors")


# ===============================
# Cell 2: HF → Tunix mapping
# ===============================
import jax
import jax.numpy as jnp
import tunix
from flax import nnx
from tunix.models.qwen3 import model as qwen_model

print("🧱 Building Tunix Qwen3-1.7B abstract model...")

model_config = qwen_model.ModelConfig.qwen3_1p7b()

abs_model: nnx.Module = nnx.eval_shape(
    lambda: qwen_model.Qwen3(
        config=model_config,
        rngs=nnx.Rngs(params=jax.random.PRNGKey(0)),
    )
)

abs_state = nnx.state(abs_model)
print("✅ Abstract model ready")

def torch_to_jax(x):
    return jnp.asarray(x.detach().cpu().numpy())

def map_hf_to_tunix(abs_state, hf_state):
    mapped = {}
    for k in abs_state:
        hf_key = (
            k.replace("layers.", "model.layers.")
             .replace("embedder.input_embedding", "model.embed_tokens.weight")
             .replace("final_norm.w", "model.norm.weight")
             .replace("lm_head.w", "lm_head.weight")
        )
        if hf_key in hf_state:
            mapped[k] = torch_to_jax(hf_state[hf_key])
    return nnx.freeze(mapped)

print("🔁 Mapping HF → Tunix weights...")
tunix_state = map_hf_to_tunix(abs_state, hf_state)
print("✅ Weight mapping complete")


# ===============================
# Cell 3: Save Tunix checkpoint
# ===============================
import orbax.checkpoint as ocp

OUT_CKPT_DIR = "/tmp/qwen3_1p7b_tunix_ckpt"
os.makedirs(OUT_CKPT_DIR, exist_ok=True)

print("💾 Saving Tunix checkpoint...")
checkpointer = ocp.StandardCheckpointer()
checkpointer.save(OUT_CKPT_DIR, tunix_state)
checkpointer.wait_until_finished()

print("🎉 DONE!")
print("📂 Tunix checkpoint saved at:", OUT_CKPT_DIR)

# Cleanup (important on Kaggle)
del hf_model, hf_state
gc.collect()

ref_model, mesh, model_config = get_qwen_ref_model(
    ckpt_path="/tmp/qwen3_1p7b_tunix_ckpt"
)

In [ ]:
# ===============================
# Load reference model
# ===============================
ref_model, mesh, model_config = get_gemma_ref_model(
    ckpt_path=CKPT_PATH
)

print("✅ Reference model loaded")

# ===============================
# Create LoRA actor
# ===============================
lora_policy = get_lora_model(ref_model, mesh)

print("✅ LoRA actor created")

# ===============================
# Cleanup memory
# ===============================
del base_model, base_state
gc.collect()

# ===============================
# Sanity check
# ===============================
actor_params = nnx.state(lora_policy)
print(f"Actor param leaves: {len(jax.tree.leaves(actor_params))}")


## Define reward functions

We define four reward functions:

- reward if the format of the output exactly matches the instruction given in
`TEMPLATE`;
- reward if the format of the output approximately matches the instruction given
in `TEMPLATE`;
- reward if the answer is correct/partially correct;
- Sometimes, the text between `<answer>`, `</answer>` might not be one
  number. So, we extract the number, and reward the model if the answer is correct.

The reward functions are inspired from
[here](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb).

First off, let's define a RegEx for checking whether the format matches.

In [ ]:
match_format = re.compile(
    rf"^[\s]{{0,}}"
    rf"{REASONING_START}.+?{REASONING_END}.*?"
    rf"{ANSWER_START}(.+?){ANSWER_END}"
    rf"[\s]{{0,}}$",
    flags=re.MULTILINE | re.DOTALL,
)

match_format.search(
    f"{REASONING_START}Let me"
    f" think!{REASONING_END}{ANSWER_START}2{ANSWER_END}",
)

Give the model a reward of 3 points if the format matches exactly.

In [ ]:
def match_format_exactly(prompts, completions, **kwargs):
  return [
      0 if match_format.search(response) is None else 3.0
      for response in completions
  ]

We also reward the model if the format of the output matches partially.

In [ ]:
def match_format_approximately(prompts, completions, answer, **kwargs):
  question = kwargs["question"]
  responses = completions

  extracted_responses = [
      guess.group(1) if (guess := match_numbers.search(r)) is not None else None
      for r in responses
  ]
  print("START ============================")
  print(f"Question: {question[0]}")
  print(f"Answer: {answer[0]}")
  print(f"Response: {responses[0]}")
  print(f"Extracted: {extracted_responses[0]}")
  print("END ==============================")
    
  scores = []
  for completion in completions:
    score = 0
    response = completion
    # Count how many keywords are seen - we penalize if too many!
    # If we see 1, then plus some points!
    score += 0.5 if response.count(REASONING_START) == 1 else -0.5
    score += 0.5 if response.count(REASONING_END) == 1 else -0.5
    score += 0.5 if response.count(ANSWER_START) == 1 else -0.5
    score += 0.5 if response.count(ANSWER_END) == 1 else -0.5
    scores.append(score)
  return scores

Reward the model if the answer is correct. A reward is also given if the answer
does not match exactly, i.e., based on how close the answer is to the correct
value.

In [ ]:
def check_answer(prompts, completions, answer, **kwargs):
    """
    Domain-aware correctness reward.
    Works with mixed-domain GRPO batches.
    """

    domains = kwargs["domain"]
    responses = completions

    # Extract answer text from <answer>...</answer>
    extracted = [
        m.group(1).strip() if (m := match_format.search(r)) else None
        for r in responses
    ]

    scores = []

    assert len(extracted) == len(answer) == len(domains), (
        f"Length mismatch: extracted={len(extracted)}, "
        f"answer={len(answer)}, domain={len(domains)}"
    )

    for guess, gold, domain in zip(extracted, answer, domains):
        # --------------------
        # Missing answer
        # --------------------
        if guess is None:
            scores.append(0.0)
            continue

        guess_l = guess.lower().strip()
        gold_l = str(gold).lower().strip()

        score = 0.0

        # ====================
        # 🔢 MATH
        # ====================
        if domain == "math":
            try:
                g = float(guess)
                a = float(gold)

                if g == a:
                    score = 3.0
                else:
                    ratio = g / a if a != 0 else 0.0
                    if 0.95 <= ratio <= 1.05:
                        score = 1.5
                    elif 0.9 <= ratio <= 1.1:
                        score = 0.5
                    else:
                        score = -1.0
            except:
                score = -0.5

        # ====================
        # 🔬 SCIENCE (MCQ)
        # ====================
        elif domain == "science":
            if guess_l == gold_l:
                score = 3.0
            else:
                score = -1.0

        # ====================
        # 🧠 LOGIC (yes / no)
        # ====================
        elif domain == "logic":
            yes_set = {"yes", "true"}
            no_set = {"no", "false"}

            if guess_l in yes_set and gold_l in yes_set:
                score = 3.0
            elif guess_l in no_set and gold_l in no_set:
                score = 3.0
            else:
                score = -1.0

        # ====================
        # 💻 CODE (handled elsewhere)
        # ====================
        elif domain == "code":
            score = 0.0  # execution reward handles this

        # ====================
        # ❓ Unknown domain
        # ====================
        else:
            score = 0.0

        scores.append(score)

    return scores

Sometimes, the text between `<answer>` and `</answer>` might not be one
number; it can be a sentence. So, we extract the number and compare the answer.

In [ ]:
match_numbers = re.compile(
    rf"{ANSWER_START}.*?([\d\.]{{1,}})", flags=re.MULTILINE | re.DOTALL
)
match_numbers.findall(f"{ANSWER_START}  0.34  {ANSWER_END}")

In [ ]:
def check_numbers(prompts, completions, answer, **kwargs):
  question = kwargs["question"]
  responses = completions

  extracted_responses = [
      guess.group(1) if (guess := match_numbers.search(r)) is not None else None
      for r in responses
  ]

  scores = []
  print("START ============================")
  print(f"Question: {question[0]}")
  print(f"Answer: {answer[0]}")
  print(f"Response: {responses[0]}")
  print(f"Extracted: {extracted_responses[0]}")
  print("END ==============================")
    
  for guess, true_answer in zip(extracted_responses, answer):
    if guess is None:
      scores.append(0)
      continue
    # Convert to numbers
    try:
      true_answer = float(true_answer.strip())
      guess = float(guess.strip())
      scores.append(1.5 if guess == true_answer else 0.0)
    except:
      scores.append(0)
      continue
  return scores

In [ ]:
def punish_refusal(prompts, completions, **kwargs):
    scores = []

    REFUSAL_PHRASES = [
        "please provide the problem",
        "i need the problem",
        "cannot solve without",
        "please provide the reasoning",
        "don’t provide",
        "i cannot help",
        "cannot answer",
        "unable to solve",
        "please provide",
        "need more information",
    ]

    for completion in completions:
        # -----------------------------
        # Safe text extraction
        # -----------------------------
        if isinstance(completion, str):
            text = completion.lower()
        elif isinstance(completion, list) and len(completion) > 0:
            first = completion[0]
            if isinstance(first, dict) and "content" in first:
                text = first["content"].lower()
            else:
                text = str(completion).lower()
        else:
            text = str(completion).lower()

        text = text.strip()

        # -----------------------------
        # 🚨 HARD REFUSAL = ABSOLUTE DEATH
        # -----------------------------
        if any(p in text for p in REFUSAL_PHRASES):
            scores.append(-20.0)   # ☢️ nuke it from orbit
            continue

        # -----------------------------
        # 🚫 Empty / ultra-short junk
        # -----------------------------
        if len(text) < 15:
            scores.append(-10.0)
            continue

        # -----------------------------
        # ✅ ATTEMPT BONUS (CRITICAL)
        # -----------------------------
        attempted = (
            "<reasoning>" in text
            or "<answer>" in text
            or any(c.isdigit() for c in text)
        )

        if attempted:
            scores.append(+1.0)   # 🛟 trying > refusing
        else:
            scores.append(-2.0)   # vague fluff still bad

    return scores


In [ ]:
def penalize_length_and_rambling(prompts, completions, **kwargs):
    scores = []

    MAX_LEN = 80          # tokens-ish proxy (chars OK too)
    LENGTH_PENALTY = 4.0  # strong on purpose

    RAMBLE_MARKERS = [
        "let's re-read",
        "however",
        "this is not correct",
        "not possible",
        "we are given that",
        "Let's rephrase the problem",
        "let us",
    ]

    for completion in completions:
        # Extract text safely
        if isinstance(completion, str):
            text = completion
        elif isinstance(completion, list) and len(completion) > 0:
            text = completion[0].get("content", "")
        else:
            text = str(completion)

        text_lower = text.lower()
        score = 0.0

        # ===============================
        # 1️⃣ Length penalty (HUGE)
        # ===============================
        length = len(text)
        if length > MAX_LEN:
            excess = (length - MAX_LEN) / MAX_LEN
            score -= LENGTH_PENALTY * excess

        # ===============================
        # 2️⃣ Talking AFTER answer (ILLEGAL)
        # ===============================
        if "</answer>" in text:
            after = text.split("</answer>", 1)[-1].strip()
            if after:
                score -= 2.5  # hard slap

        # ===============================
        # 3️⃣ Rambling / restart detection
        # ===============================
        ramble_hits = sum(m in text_lower for m in RAMBLE_MARKERS)
        score -= 0.5 * ramble_hits

        scores.append(score)

    return scores

In [ ]:
import re
from collections import Counter

# 🔑 Generic reasoning markers (domain-agnostic)
REASONING_KEYWORDS = [
    # logical flow
    "because", "therefore", "thus", "hence", "so", "as a result",
    "this implies", "it follows", "which means",

    # reasoning actions
    "assume", "consider", "analyze", "evaluate", "compare",
    "explain", "reason", "conclude", "determine",

    # structure
    "first", "second", "next", "then", "finally",

    # evidence / grounding
    "given", "based on", "from this", "according to"
]


def reasoning_quality_reward(prompts, completions, **kwargs):
    scores = []

    for response in completions:
        score = 0.0
        text = response.lower()

        # -----------------------------------
        # 1️⃣ Require reasoning block
        # -----------------------------------
        if "<reasoning>" not in text or "</reasoning>" not in text:
            scores.append(-0.4)
            continue

        m = re.search(r"<reasoning>(.*?)</reasoning>", text, re.S)
        if m is None:
            scores.append(-0.4)
            continue

        reasoning = m.group(1).strip()

        # -----------------------------------
        # 2️⃣ Sentence structure (domain-agnostic)
        # -----------------------------------
        sentences = [
            s.strip() for s in re.split(r"[.\n]", reasoning)
            if len(s.strip()) > 6
        ]

        if len(sentences) >= 2:
            score += 0.15
        if len(sentences) >= 4:
            score += 0.15
        if len(sentences) >= 7:
            score += 0.1

        # -----------------------------------
        # 3️⃣ Reasoning keyword usage (NOT spam)
        # -----------------------------------
        keyword_hits = sum(reasoning.count(k) for k in REASONING_KEYWORDS)

        if 1 <= keyword_hits <= 5:
            score += 0.25
        elif keyword_hits > 8:
            score -= 0.25  # keyword spam

        # -----------------------------------
        # 4️⃣ Keyword repetition penalty (ONLY keywords)
        # -----------------------------------
        keyword_counts = Counter()

        for kw in REASONING_KEYWORDS:
            c = reasoning.count(kw)
            if c > 0:
                keyword_counts[kw] += c

        if keyword_counts:
            max_rep = max(keyword_counts.values())

            if max_rep >= 5:
                score -= 0.5
            elif max_rep == 4:
                score -= 0.35
            elif max_rep == 3:
                score -= 0.2
            elif max_rep == 2:
                score -= 0.1

        # -----------------------------------
        # 5️⃣ Length sanity (GENERIC, relaxed)
        # -----------------------------------
        token_len = len(reasoning.split())

        if token_len < 25:
            score -= 0.25
        elif 50 <= token_len <= 250:
            score += 0.25
        elif 250 < token_len <= 450:
            score += 0.15
        elif token_len > 600:
            score -= 0.35  # rambling

        # -----------------------------------
        # 6️⃣ Grounding signals (numbers OR entities OR examples)
        # -----------------------------------
        has_numbers = bool(re.search(r"\d", reasoning))
        has_examples = "example" in reasoning or "for instance" in reasoning
        has_entities = bool(re.search(r"[A-Z][a-z]+", m.group(1)))

        grounding_hits = sum([has_numbers, has_examples, has_entities])

        if grounding_hits >= 1:
            score += 0.15
        if grounding_hits >= 2:
            score += 0.15

        # -----------------------------------
        # 7️⃣ Penalize pure fluff phrases
        # -----------------------------------
        fluff_phrases = [
            "it is obvious", "clearly", "everyone knows",
            "needless to say", "without loss of generality"
        ]

        if any(p in reasoning for p in fluff_phrases):
            score -= 0.3

        # -----------------------------------
        # 8️⃣ Final clamp (keep GRPO stable)
        # -----------------------------------
        score = max(-0.6, min(0.9, score))
        scores.append(score)

    return scores


## Evaluate


Before we train the model, let's evaluate the model on the test set so we can
see the improvement post training.

We evaluate it in two ways:

**Quantitative**

* **Answer Accuracy**: percentage of samples for which the model predicts the
correct final numerical answer  
* **Answer (Partial) Accuracy**: percentage of samples for which the model
predicts a final numerical answer such that the \`model answer / answer\`
ratio lies between 0.9 and 1.1.  
* **Format Accuracy**: percentage of samples for which the model outputs the
correct format, i.e., reasoning between the reasoning special tokens, and the
final answer between the \`\<start\_answer\>\`, \`\<end\_answer\>\` tokens.

**Qualitative**

We'll also print outputs for a few given questions so that we can compare the generated output later.


We define a helper function to generate an answer, given a prompt.

In [ ]:
def generate(
    question, sampler, temperature=0.7, top_k=50, top_p=0.95, seed=None
):
  """Given prompt, generates text."""

  if isinstance(question, str):
    input_batch = [
        TEMPLATE.format(
            system_prompt=SYSTEM_PROMPT,
            question=question,
        ),
    ]
  else:
    input_batch = [
        TEMPLATE.format(
            system_prompt=SYSTEM_PROMPT,
            question=q,
        )
        for q in question
    ]

  out_data = sampler(
      input_strings=input_batch,
      max_generation_steps=768,
      temperature=temperature,
      top_k=top_k,
      top_p=top_p,
      echo=False,
      seed=seed if seed is not None else None,
      eos_tokens=[1,106],
  )

  output = out_data.text
  if isinstance(question, str):
    return output[0]
  return output
    

Another helper function for evaluation.

In [ ]:
def evaluate(
    dataset,
    sampler,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_passes=1,
    corr_lst=False,
    make_lst=False,
):
  """Computes accuracy and percentage of outputs matching the format."""

  response_lst = []
  corr = 0
  partially_corr = 0
  corr_format = 0
  total = 0

  for batch in tqdm(dataset):
    answers = batch["answer"]
    questions = batch["question"]

    multiple_call_responses = [[] for _ in range(len(questions))]
    for p in range(num_passes):
      responses = generate(
          questions, sampler, temperature, top_k, top_p, seed=p
      )
      for idx, response in enumerate(responses):
        multiple_call_responses[idx].append(response)

    for question, multiple_call_response, answer in zip(
        questions, multiple_call_responses, answers
    ):
      # check answer
      corr_ctr_per_question = 0
      partially_corr_per_question = 0
      corr_format_per_question = 0
      for response in multiple_call_response:
        extracted_response = (
            guess.group(1)
            if (guess := match_numbers.search(response)) is not None
            else "-1000000"
        )
        try:
          if float(extracted_response.strip()) == float(answer.strip()):
            corr_ctr_per_question += 1

          ratio = float(extracted_response.strip()) / float(answer.strip())
          if ratio >= 0.9 and ratio <= 1.1:
            partially_corr_per_question += 1
        except:
          print("SKIPPED")

        # check format
        if match_format.search(response) is not None:
          corr_format_per_question += 1

        if (
            corr_ctr_per_question > 0
            and partially_corr_per_question > 0
            and corr_format_per_question > 0
        ):
          break

      if corr_ctr_per_question > 0:
        corr += 1
        if corr_lst and make_lst:
          response_lst.append((question, answer, multiple_call_response))
      else:
        if not corr_lst and make_lst:
          response_lst.append((question, answer, multiple_call_response))
      if partially_corr_per_question > 0:
        partially_corr += 1
      if corr_format_per_question > 0:
        corr_format += 1

      total += 1
      if total % 10 == 0:
        print(
            f"===> {corr=}, {total=}, {corr / total * 100=}, "
            f"{partially_corr / total * 100=}, {corr_format / total * 100=}"
        )

  to_return = (
      corr,
      total,
      corr / total * 100,
      partially_corr / total * 100,
      corr_format / total * 100,
  )
  if make_lst:
    return to_return, response_lst
  return to_return

In [ ]:
sampler = sampler_lib.Sampler(
    transformer=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

Now let's see how the original model does on the test set. You can see the percentages of the mode outputs that are fully correct, partially correct and just correct in format. The following step might take couple of minutes to finish.

In [ ]:
# The evaluation might take up to couple of minutes to finish. Please be patient.

(corr, total, accuracy, partial_accuracy, format_accuracy) = evaluate(
    test_dataset,
    sampler,
    **GENERATION_CONFIGS["greedy"],
)
print(
    f"{corr=}, {total=}, {accuracy=}%, {partial_accuracy=}%,"
    f" {format_accuracy=}%"
)

## Train

Let's set up all the configs first - checkpointing, metric logging and training.
We then train the model.

In [ ]:
# ===============================
# Checkpointing options
# ===============================
checkpointing_options = ocp.CheckpointManagerOptions(
    save_interval_steps=SAVE_INTERVAL_STEPS,
    max_to_keep=MAX_TO_KEEP,
)

# ===============================
# Metrics logger options (NEW API)
# ===============================
metrics_logging_options = metrics_logger.MetricsLoggerOptions(
    log_dir="/tmp/content/tmp/tensorboard/grpo",
    project_name="tunix-grpo",
    run_name="gemma3-1b-grpo",
    flush_every_n_steps=20,
)


In [ ]:
# Optimizer, learning rate scheduler, gradient clipping
optimizer = optax.adamw(
    learning_rate=optax.schedules.warmup_cosine_decay_schedule(
        init_value=0.0,
        peak_value=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        decay_steps=MAX_STEPS,
        end_value=0.0,
    ),
    b1=B1,
    b2=B2,
    weight_decay=WEIGHT_DECAY,
)
if MAX_GRAD_NORM is not None:
  optimizer = optax.chain(
      optax.clip_by_global_norm(max_norm=MAX_GRAD_NORM),
      optimizer,
  )
    

In [ ]:
# Training config
cluster_config = rl_cluster_lib.ClusterConfig(
    role_to_mesh={
        rl_cluster_lib.Role.ACTOR: mesh,
        rl_cluster_lib.Role.REFERENCE: mesh,
        rl_cluster_lib.Role.ROLLOUT: mesh,
    },
    rollout_engine='vanilla',
    offload_to_cpu=False,
    training_config=rl_cluster_lib.RLTrainingConfig(
        actor_optimizer=optimizer,
        eval_every_n_steps=EVAL_EVERY_N_STEPS,
        max_steps=MAX_STEPS,
        mini_batch_size=TRAIN_MICRO_BATCH_SIZE,
        train_micro_batch_size=TRAIN_MICRO_BATCH_SIZE,
        # metrics logging
        metrics_logging_options=metrics_logging_options,
        # checkpoint saving
        checkpoint_root_directory=CKPT_DIR,
        checkpointing_options=checkpointing_options,
    ),
    rollout_config=base_rollout.RolloutConfig(
        max_tokens_to_generate=TOTAL_GENERATION_STEPS,
        max_prompt_length=MAX_PROMPT_LENGTH,
        kv_cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        top_k=TOP_K,
        eos_tokens=[1,106],
    ),
)
grpo_config = GRPOConfig(
    num_generations=NUM_GENERATIONS,
    num_iterations=NUM_ITERATIONS,
    beta=BETA,
    epsilon=EPSILON,
)

### Setting Up the GRPO Trainer

Now we initialize our system for training. First, we create an `RLCluster` instance, which brings together the **policy model (`actor`)**, a **reference model (`reference`)**, and a **tokenizer**. Our `actor` is a trainable LoRA model, while the `reference` is a fixed base model that we use to guide the training.

We then create a `GRPOLearner`, the specialized trainer that uses a list of **reward functions** to evaluate and optimize the model's output, completing the RL training setup.

Tunix trainers are integrated with [Weights & Biases](https://wandb.ai/) to help you visualize the training progress. You can choose how you want to use it:

**Option 1 (Type 1)**: If you're running a quick experiment or just testing things out, choose this. It creates a temporary, private dashboard right in your browser without requiring you to log in or create an account.

**Option 2 (Type 2)**: If you have an existing W&B account and want to save your project's history to your personal dashboard, choose this. You'll be prompted to enter your API key or log in.

In [ ]:
# RL cluster
rl_cluster = rl_cluster_lib.RLCluster(
    actor=lora_policy,
    reference=ref_model,
    tokenizer=tokenizer,
    cluster_config=cluster_config,
)
# GRPO Trainer
grpo_trainer = GRPOLearner(
    rl_cluster=rl_cluster,
    reward_fns = [
        punish_refusal,             # 🚨 hard negative
        match_format_exactly,       # 🚪 gate
        match_format_approximately, # 🧭 soft format
        check_answer,               # 🎯 correctness
        penalize_length_and_rambling # 🔥 TERMINATION PRESSURE
    ],
    algo_config=grpo_config,
)

The first couple of training step might take up to 5 minutes to finish. Please be patient. If you experience long training steps, e.g. >10 minutes per step, please open a bug. Really appreciated!


---

# 🧠 Training Strategy: Phases, Domains, and Rewards (Tunix + Gemma)

This project trains a **Gemma3-1B + LoRA** model using **GRPO (Tunix)** with a **curriculum-based reinforcement learning strategy**.
The key idea is to **separate behavioral alignment from task competence**, and only introduce domain complexity after the model reliably follows the output contract.

---

## 🎯 Core Principle

> **Reinforcement Learning teaches behavior, not skills.**
> Skills come from the base model and data; RL enforces *how* the model responds.

Therefore:

* Early phases focus on **format, discipline, and attempt behavior**
* Later phases introduce **correctness**
* Domain diversity is introduced **only after behavior stabilizes**

---

## 🥇 Phase 1 — Behavioral Alignment (Format & Discipline)

### 🎯 Objective

Train the model to:

* Always attempt an answer
* Always output **only** inside:

  ```text
  <reasoning>...</reasoning>
  <answer>...</answer>
  ```
* Never refuse
* Avoid rambling or extraneous text

**Correctness is NOT optimized in this phase.**

---

### 📚 Dataset

* **GSM8K (subset)**
* Random 20–30% slice, or filtered for short questions

Reason:

* GSM8K prompts are short and structured
* Easy to parse
* Domain content is irrelevant at this stage

---

### 🏆 Reward Functions (Phase 1)

```python
reward_fns = [
    punish_refusal,        # hard negative for refusing / dodging
    strict_format_gate,    # requires valid <answer> tag
    light_length_penalty   # discourages rambling
]
```

**No correctness rewards.**
**No numeric checks.**
**No partial credit.**

Format is a **gate**, not a prize.

---

### 📊 Metrics to Track

* **Answer extraction rate** (primary KPI)
* Refusal rate
* Average output length

### ✅ Phase Completion Criteria

* ≥ 95% successful `<answer>` extraction
* Near-zero refusal
* Stable output length

---

## 🥈 Phase 2 — Correctness & Honesty (Single Domain)

### 🎯 Objective

Teach the model:

* Be correct when possible
* Be concise when wrong
* Do not hallucinate

---

### 📚 Dataset

* **Full GSM8K**

The domain stays fixed to avoid confounding behavior with task switching.

---

### 🏆 Reward Functions (Phase 2)

```python
reward_fns = [
    punish_refusal,
    strict_format_gate,
    check_answer_strict,    # exact match only (no ratios)
    penalize_length         # stronger penalty for verbose wrong answers
]
```

Design choices:

* Binary correctness signal
* No ratio-based partial credit
* Wrong answers are penalized, especially if verbose

---

### 📊 Metrics to Track

* Answer extraction rate (should remain high)
* Hallucination frequency
* Average length of wrong answers
* Accuracy trend (expected to improve slowly)

---

## 🟦 Phase 3 — Domain Generalization (Optional but Recommended)

### 🎯 Objective

Apply the learned **behavioral contract** across domains:

* Code
* Creative reasoning
* Science
* Open-ended QA

---

### 📚 Dataset

A **mixed dataset**, e.g.:

| Domain         | Proportion |
| -------------- | ---------- |
| GSM8K          | 50%        |
| Code           | 25%        |
| QA / Reasoning | 25%        |

---

### 🏆 Reward Functions (Phase 3)

```python
reward_fns = SAME AS PHASE 2
```

**Important rule:**

> **Do NOT change rewards when introducing new domains.**

Consistency ensures behavior generalizes.

---

### 📊 Metrics to Track

* Format compliance across domains
* Refusal rate
* Rambling frequency
* Qualitative reasoning quality

---

## 🔁 Phase Transitions (Rules)

* ❌ Do NOT change dataset and rewards at the same time
* ❌ Do NOT mix domains before Phase 1 stabilizes
* ✅ Only advance phases after behavior plateaus

---

## 🧠 Why This Works for Tunix Evaluation

* Enforces the exact output format required by judges
* Aligns with human + LLM-as-judge evaluation
* Avoids over-optimizing math (low eval weight)
* Produces consistent, interpretable reasoning traces

---

## 🏁 Summary

| Phase   | Focus               | Dataset       | Rewards                      |
| ------- | ------------------- | ------------- | ---------------------------- |
| Phase 1 | Format & discipline | GSM8K subset  | Format + refusal + brevity   |
| Phase 2 | Correctness         | Full GSM8K    | Strict correctness + brevity |
| Phase 3 | Generalization      | Mixed domains | Same as Phase 2              |

---

> **Final takeaway:**
> First teach the model *how to behave*.
> Only then ask it to be right — everywhere.

---

If you want next:

* 📈 a **phase auto-switching heuristic**
* 🧪 a **live extraction-rate logger**
* 🧠 a **judge-aligned evaluation prompt**

say the word 😤🔥
def iterable_slice(dataset, start_frac, end_frac):
    N = len(dataset)
    start = int(start_frac * N)
    end = int(end_frac * N)

    def generator():
        for i, sample in enumerate(dataset):
            if i >= end:
                break
            if i >= start:
                yield sample

    return generator()


In [ ]:
with mesh:
  grpo_trainer.train(train_dataset)

## Evaluate

Let's evaluate our finetuned model!

In [ ]:
# Load checkpoint first.
import re

# Find the latest checkpoint by listing directories in CKPT_DIR/actor
actor_ckpt_dir = os.path.join(CKPT_DIR, "actor")

latest_step = -1
if os.path.exists(actor_ckpt_dir):
  for item in os.listdir(actor_ckpt_dir):
    if os.path.isdir(os.path.join(actor_ckpt_dir, item)) and re.match(r'^\d+$', item):
      step = int(item)
      if step > latest_step:
        latest_step = step

if latest_step == -1:
  raise FileNotFoundError(f"No checkpoints found in {actor_ckpt_dir}")

print(f"Latest checkpoint step: {latest_step}")

wandb.init(project='tunix-eval')  # logging bug workaround

trained_ckpt_path = os.path.join(
    CKPT_DIR, "actor", str(latest_step), "model_params"
)

abs_params = jax.tree.map(
    lambda x: jax.ShapeDtypeStruct(x.shape, x.dtype),
    nnx.state(lora_policy, nnx.LoRAParam),
)
checkpointer = ocp.StandardCheckpointer()
trained_lora_params = checkpointer.restore(trained_ckpt_path, target=abs_params)

nnx.update(
    lora_policy,
    jax.tree.map(
        lambda a, b: b,
        nnx.state(lora_policy, nnx.LoRAParam),
        trained_lora_params,
    ),
)

In [ ]:
sampler = sampler_lib.Sampler(
    transformer=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

In [ ]:
# The evaluation might take up to couple of minutes to finish. Please be patient.
(corr, total, accuracy, partial_accuracy, format_accuracy) = evaluate(
    test_dataset,
    sampler,
    **GENERATION_CONFIGS["greedy"],
)
print(
    f"{corr=}, {total=}, {accuracy=}%, {partial_accuracy=}%,"
    f" {format_accuracy=}%"
)

With sufficient training, you should see that the percentages of correct model outputs have clearly gone up, which means our training worked.

In [ ]:
# =====================================
# SAVE FULL TUNIX-LOADABLE CHECKPOINT
# =====================================
import os
import json
import orbax.checkpoint as ocp
from flax import nnx

EXPORT_DIR = "./tunix_full_model"
os.makedirs(EXPORT_DIR, exist_ok=True)

# Extract FULL params (base + LoRA merged)
full_params = nnx.state(lora_policy)

# Save using Orbax (Tunix-compatible)
full_checkpointer = ocp.StandardCheckpointer()
full_checkpointer.save(
    EXPORT_DIR,
    full_params,
    force=True,
)

# Optional but recommended: minimal model card
model_card = {
    "model_name": "gemma3-1b-grpo-reasoning",
    "base_model": "google/gemma-3-1b",
    "framework": "tunix / jax / flax-nnx",
    "training_method": "GRPO",
    "task": "reasoning"
}

with open(os.path.join(EXPORT_DIR, "model_card.json"), "w") as f:
    json.dump(model_card, f, indent=2)

print("✅ Full Tunix model saved at:", EXPORT_DIR)

In [ ]:
# =====================================
# UPLOAD FULL MODEL TO KAGGLE (MODEL)
# =====================================

import json
import subprocess

MODEL_SLUG = "gemma3-1b-grpo-reasoning-tunix"

OWNER = subprocess.check_output(["kaggle", "whoami"]).decode().strip()
MODEL_ID = f"{OWNER}/{MODEL_SLUG}"

print(f"📦 Uploading Kaggle model: {MODEL_ID}")

# Kaggle model metadata
metadata = {
    "title": "Gemma3 1B GRPO Reasoning (Tunix)",
    "id": MODEL_ID,
    "licenses": [{"name": "apache-2.0"}]
}

with open("model-metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

# Create model (no-op if already exists)
!kaggle models create -p . --metadata model-metadata.json || true

# Upload model version
!kaggle models versions create \
    -p tunix_full_model \
    -m "Full Gemma3 1B model trained with GRPO (Tunix compatible)"

print("🎉 DONE")
print("✅ SUBMIT THIS MODEL NAME:")
print(MODEL_ID)